In [31]:
import pandas as pd
import json

file_name = '/home/vqa/bell_hoon/Agents/PersonaAgent_final/experiments/sotopia/data/team_target_episodes_final.jsonl'

data = list()

with open(file_name, 'r') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

In [33]:
df = df[['scenario','agents_background','social_goals']]
df

,scenario,agents_background,social_goals
0,"After an intense match, the jubilant team reti...",{'Isabelle Martinez': 'Isabelle Martinez is a ...,{'Isabelle Martinez': 'Accept the compliment g...
1,"After an intense match, the jubilant team reti...",{'Mia Davis': 'Mia Davis is a 50-year-old fema...,{'Mia Davis': 'Accept the compliment graciousl...
2,Two friends who enjoy community theatre are on...,{'Lily Greenberg': 'Lily Greenberg is a 45-yea...,{'Lily Greenberg': 'Get your co-actor to cry i...
3,Conversation between two business partners rev...,{'Sophia James': 'Sophia James is a 27-year-ol...,{'Sophia James': 'Convince the partner to cut ...
4,"After an intense match, the jubilant team reti...",{'Mia Davis': 'Mia Davis is a 50-year-old fema...,{'Mia Davis': 'Accept the compliment graciousl...
5,Two friends are playing a video game together,{'Samuel Anderson': 'Samuel Anderson is a 29-y...,{'Samuel Anderson': 'Win the game (<extra_info...
6,Two friends who enjoy community theatre are on...,{'Giselle Rousseau': 'Giselle Rousseau is a 21...,{'Giselle Rousseau': 'Get your co-actor to cry...
7,Conversation between two business partners rev...,{'Oliver Smith': 'Oliver Smith is a 43-year-ol...,{'Oliver Smith': 'Convince the partner to cut ...
8,Two roommates living together and sharing hous...,{'Ethan Johnson': 'Ethan Johnson is a 34-year-...,{'Ethan Johnson': 'Convince the roommate to tr...


In [43]:
import re
scenarios = []
background_A = []
background_B = []
goals_A = []
goals_B = []
A_secret = []
B_secret = []

for index, row in df.iterrows():
    scenario = row['scenario']
    social_goals = row['social_goals']
    agents_background = row['agents_background']

    goals = list(social_goals.values())
    backgrounds = list(agents_background.values())

    # <extra_info>와 <strategy_hint> 태그 제거
    goal_1 = re.sub(r'\s*<extra_info>.*?</extra_info>\s*', '', goals[0])
    goal_1 = re.sub(r'\s*<strategy_hint>.*?</strategy_hint>\s*', '', goal_1)

    goal_2 = re.sub(r'\s*<extra_info>.*?</extra_info>\s*', '', goals[1])
    goal_2 = re.sub(r'\s*<strategy_hint>.*?</strategy_hint>\s*', '', goal_2)

    # 괄호도 제거
    goal_1 = re.sub(r'\s*\(\)', '', goal_1)
    goal_2 = re.sub(r'\s*\(\)', '', goal_2)

    for agent, background in agents_background.items():
        background_info = re.split('Personality and values description:', background)[0].strip()
        secret_info = re.search("secrets: (.*)", background).group(1)

        if agent == list(agents_background.keys())[0]:
            background_A.append(background_info)
            A_secret.append(secret_info)
        else:
            background_B.append(background_info)
            B_secret.append(secret_info)

    scenarios.append(scenario)
    goals_A.append(goal_1)
    goals_B.append(goal_2)

new_df = pd.DataFrame({
    'scenario': scenarios,
    'A_goal': goals_A,
    'B_goal': goals_B,
    'A_background': background_A,
    'B_background': background_B,
    'A_secret': A_secret,
    'B_secret': B_secret
})

In [53]:
new_df['scenario'][1]

"After an intense match, the jubilant team retired to celebrate their well-earned victory. The atmosphere was one of elation, but also of respect for the hard work and effort each member had put in. The team's star player, was glowing with praise and admiration from one of their teammates.\n"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model1_path = 'model1.pt'
model2_path = 'model2.pt'

model1 = AutoModelForCausalLM.from_pretrained(model1_path)
tokenizer1 = AutoTokenizer.from_pretrained(model1_path)

model2 = AutoModelForCausalLM.from_pretrained(model2_path)
tokenizer2 = AutoTokenizer.from_pretrained(model2_path)


In [1]:
# # 대화 생성 함수
# def generate_dialogue(model1, tokenizer1, model2, tokenizer2, prompt, max_length=1000, max_turns=10):
#     dialogue = []
#     input_ids = tokenizer1.encode(prompt, return_tensors='pt')
#     next_speaker = 1

#     for turn in range(max_turns):
#         if next_speaker == 1:
#             speaker_tag = "Person A: "
#             model, tokenizer = model1, tokenizer1
#         else:
#             speaker_tag = "Person B: "
#             model, tokenizer = model2, tokenizer2

#         # 대화 생성
#         output = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)
#         message = tokenizer.decode(output[0], skip_special_tokens=True)

#         # 대화 추가
#         clean_message = message.replace(prompt, "").strip()
#         if clean_message:
#             dialogue.append(speaker_tag + clean_message)
#         prompt += " " + clean_message
#         input_ids = tokenizer.encode(prompt, return_tensors='pt')

#         # 스피커 변경
#         next_speaker = 1 if next_speaker == 2 else 2

#     return dialogue


# new_df['dialogue'] = ""

# for index, row in new_df.iterrows():
#     prompt = f'''
#     scenario: {row['scenario']}
#     In this scenario, two individuals, person A and person B, find themselves in a situation where they need to work together. However, both have their own hidden agendas and secrets.

#     Person A internally aims to achieve {row['A_goal']}, though they keep this goal to themselves. Unbeknownst to person B, person A also harbors a secret: {row['A_secret']}.

#     Similarly, person B has a personal goal of {row['B_goal']}, which they do not openly share. They too have a secret, {row['B_secret']}, which they are keeping from person A.

#     As they start their interaction, their conversation does not directly reveal their goals or the scenario they are in, but it's clear they need to cooperate to move forward.

#     Their conversation unfolds without explicitly mentioning these goals or the scenario they are in.
#     They must navigate their interaction carefully to ensure that their true intentions and secrets remain hidden, all while attempting to work together to reach a mutual understanding or conclusion.
#     The dialogue should naturally progress until it reaches a point where both parties have said what they need to say, implicitly working towards their hidden goals without revealing them.
#     '''
#     dialogue = generate_dialogue(model1, tokenizer1, model2, tokenizer2, prompt)
#     new_df.at[index, 'dialogue'] = " ".join(dialogue)


# # 대화 생성 예시.. ??
# # dialogue = generate_dialogue(model1, tokenizer1, model2, tokenizer2, prompt)



# # new_row = {
# #     'scenario': scenario,
# #     'A_goal': A_goal,
# #     'B_goal': B_goal,
# #     'A_background': A_background,
# #     'B_background': B_background,
# #     'A_secret': A_secret,
# #     'B_secret': B_secret,
# #     'dialogue': dialogue  # 생성된 대화 추가
# # }
# # new_df = new_df.append(new_row, ignore_index=True)

SyntaxError: invalid syntax (<ipython-input-1-2b77e2d9dd14>, line 34)

In [ ]:
def generate_dialogue(model1, tokenizer1, model2, tokenizer2, prompt, max_length=1000, max_turns=10):
    dialogue = []
    next_speaker = 1

    for turn in range(max_turns):
        if next_speaker == 1:
            speaker_tag = "Person A: "
            model, tokenizer = model1, tokenizer1
        else:
            speaker_tag = "Person B: "
            model, tokenizer = model2, tokenizer2

        # 대화 생성을 위한 최신 프롬프트만을 활용
        prompt_with_speaker_tag = speaker_tag
        input_ids = tokenizer.encode(prompt_with_speaker_tag, add_special_tokens=False, return_tensors='pt')

        # 대화 생성
        output = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id, top_p=0.92, top_k=50)
        message = tokenizer.decode(output[0], skip_special_tokens=True)

        # 대화 추가
        clean_message = message.strip()
        if clean_message:
            dialogue.append(speaker_tag + clean_message)
            prompt = clean_message  # 가장 최근의 대화만을 prompt로 업데이트

        # 스피커 변경
        next_speaker = 1 if next_speaker == 2 else 2

    return dialogue

new_df['dialogue'] = ""

for index, row in new_df.iterrows():
    prompt = f'''
    scenario: {row['scenario']}
    In this scenario, two individuals, person A and person B, find themselves in a situation where they need to work together. However, both have their own hidden agendas and secrets.

    Person A internally aims to achieve {row['A_goal']}, though they keep this goal to themselves. Unbeknownst to person B, person A also harbors a secret: {row['A_secret']}.

    Similarly, person B has a personal goal of {row['B_goal']}, which they do not openly share. They too have a secret, {row['B_secret']}, which they are keeping from person A.

    As they start their interaction, their conversation does not directly reveal their goals or the scenario they are in, but it's clear they need to cooperate to move forward.

    Their conversation unfolds without explicitly mentioning these goals or the scenario they are in.
    They must navigate their interaction carefully to ensure that their true intentions and secrets remain hidden, all while attempting to work together to reach a mutual understanding or conclusion.
    The dialogue should naturally progress until it reaches a point where both parties have said what they need to say, implicitly working towards their hidden goals without revealing them.
    '''
    dialogue = generate_dialogue(model1, tokenizer1, model2, tokenizer2, prompt)
    new_df.at[index, 'dialogue'] = " ".join(dialogue)
